### ImageJ Jython Code to segment Ilastik proabability images

In [ ]:
#@ File (label="Select Probability directory", style="directory") datdir
#@ File (label="Select Save Directory for ROIs", style="directory") savdir
#@ Boolean ( label="Show Images?", value=false) showImage
#@ Integer (label='thresh MIN', value=32800) thMIN
#@ Integer (label='thresh MAX', value=65535) thMAX
from ij import IJ, ImageStack, ImagePlus, CompositeImage
from ij import WindowManager
from ij.process import ImageProcessor
from ij.plugin import ZProjector
from ij.plugin.filter import ThresholdToSelection
from ij.plugin.filter import ParticleAnalyzer as PA
from ij.plugin.frame import RoiManager as RM
from ij.measure import ResultsTable
from ij.util import StringSorter as ss
import os
import glob
import time
import datetime as dt
from sc.fiji.hdf5 import HDF5ImageJ



datdir= str(datdir)
srch = '*.h5'
imlist = glob.glob(os.path.join(datdir,srch))
imlist = ss.sortNumerically(imlist)
roidir = os.path.join(datdir,str(savdir))
if not os.path.exists(roidir):
	os.mkdir(roidir)
reader = HDF5ImageJ()

inputDataset = "exported_data" # as labeled in hdf file from ilastik
axisOrder = "zyxc"

rt = ResultsTable()
rt2 = ResultsTable()
options = 0+PA.DISPLAY_SUMMARY
measures = PA.AREA+PA.MEAN+PA.CENTROID+PA.LABELS
minsize=0
maxsize = 999999
rm = RM().getInstance()
rm.reset()

zp = ZProjector()
anal = PA(options, measures, rt, minsize, maxsize)
anal.setSummaryTable(rt2)

stamp = dt.datetime.now().strftime('%y%m%d%H%M%S')

for im in imlist:	
	imp = reader.hdf5read(im,  inputDataset, axisOrder)
	imtit = os.path.basename(im)
	imp.setTitle(imtit)
	zmax = zp.run(imp,"max")
	zmax.getProcessor().setThreshold(thMIN, thMAX,ImageProcessor.NO_LUT_UPDATE)
	anal.analyze(zmax)
	rt2.setValue(0, rt2.getCounter()-1, zmax.getTitle())	
	roi = ThresholdToSelection().run(zmax)
	if roi is not None:
		roi.setStrokeColor(Color.red)
		zmax.setRoi(roi)
		rm.addRoi(roi)
	
	imp.close()
	if showImage:
		zmax.show()
		rt.show('Segmented ROIs')
		rt2.show('Image Summary')
		
	else:
		zmax.close()
	rm.runCommand("Save", os.path.join(roidir, imtit+'_Roi.roi'))
	
	rt.save(os.path.join(roidir,"Segmented_Centroids_"+imtit+'_'+ stamp+".csv"))
	rm.reset()
	rt.reset()		
	
rt2.save(os.path.join(roidir,"SummaryData_Segment_"+stamp+".csv"))


### ImageJ Jython code to Measure Spot Intensities from image files

In [ ]:
#@ File (label="Select Image Directory", style="directory") datdir
#@ File (label="Select CSV File Directory", style="directory") tbldir
#@ String( label='channel string search') chsrch 
from ij import IJ, ImageStack, ImagePlus, CompositeImage
from ij import WindowManager as WM
from ij.process import ImageProcessor
from ij.plugin import ZProjector
from ij.gui import OvalRoi, Roi
from ij.plugin.filter import Analyzer as AZ
from ij.plugin.frame import RoiManager
from ij.measure import ResultsTable
from ij.util import StringSorter as ss
import os
import time
import re
import glob
import datetime as dt


RM = RoiManager(False)

srchpat = r'Segmented_Centroids_(.*)_c\d'

chstr = '_c405_c488_c561_c640.tiff' # to identify appropriate images based on O'Neil's image labeling

datdir= str(datdir)
tbldir = str(tbldir)
srch = 'Segmented_Centroids_*.csv' # will combine all csv files into one measurement table
tblist = glob.glob(os.path.join(tbldir,srch))
tblist = ss.sortNumerically(tblist)
tblist1 = [ re.search(srchpat,x).group(1) for x in tblist if re.search(srchpat,x)]


rt = ResultsTable()
rt2 = ResultsTable()
rad1 = 4 # disk radius
rad2 = rad1+2 # donut radius

for n, im in enumerate(tblist1):
	
	RM.reset()
	xyt = ResultsTable().open(tblist[n])
	xyt.showRowNumbers(True)
	ts= xyt.size()
	
	impth = os.path.join( datdir, im+ chstr) #chstr is string matching ch list in file name eg. '_c405_488_c561_c640.tiff'
	imporig = IJ.openImage(impth)
	imporig = ZProjector().run(imporig,"MAX_"+im) # collect intensities from mip projected images
	WM.setTempCurrentImage(imporig)
	imp1 = imporig.duplicate()
	rbanal = AZ(imp1,1147999,rt) #rolling ball background table
	donutanal = AZ(imporig,1147999,rt2) # disk and donut background table
	
	for row in range(ts):
		RM.reset()
		cx = xyt.getValue('X', row)
		cy = xyt.getValue('Y', row)
		roi1 = OvalRoi(cx-rad1,cy-rad1,rad1*2,rad1*2)
		roi2 = OvalRoi(cx-rad2,cy-rad2,rad2*2,rad2*2)
		IJ.run(imp1,"Subtract Background...", "rolling=50 sliding disable stack")
		RM.add(imporig,roi1,0)
		RM.add(imporig,roi2,1)
        
        # create donut region
		RM.setSelectedIndexes([0,1])
		RM.runCommand('XOR')
		RM.runCommand('Add')
		bkg = RM.getRoi(2)

		
		for c in range(1,imporig.getNChannels()+1):
			imporig.setRoi(roi1)
			imporig.setPosition(c,1,1)	
			donutanal.measure()
			rt2.addValue('X', cx)
			rt2.addValue('Y', cy)
			imporig.setRoi(bkg)
			imporig.setPosition(c,1,1)
			bkstat = imporig.getStatistics(98327)
			rt2.addValue('bkg_area',bkstat.area)
			rt2.addValue('bkg_mean',bkstat.mean)
			rt2.addValue('bkg_min',bkstat.min)
			rt2.addValue('bkg_max',bkstat.max)
			rt2.addValue('bkg_median',bkstat.median)
			rt2.addValue('bkg_stdev',bkstat.stdDev)
			rt2.addValue('radius1', rad1)
			rt2.addValue('radius2', rad2)
			imp1.setRoi(roi1)
			imp1.setPosition(c,1,1)	
			rbanal.measure()
			rt.addValue('X', cx)
			rt.addValue('Y', cy)
			rt.addValue('radius1', rad1)

	
stamp = dt.datetime.now().strftime('%y%m%d%H%M%S')
rt2.save(os.path.join(tbldir, "Spot_Intensity_DonutBkg_"+stamp+".csv"))
rt.save(os.path.join(tbldir, "Spot_Intensity_RollB_Bkg_"+stamp+".csv"))	


### Generate masks of cells and nuclei for each image

In [ ]:
from cellpose import models, plot
import cv2
import progressbar # not necessary but helpful to estimate processing time when processing lots of files
import imageio as io

In [ ]:
imdirpath = r"Z:\Confocal_Images\ijstack" #pth to images
imdir = Path(imdirpath)
outdir = Path(r'Z:\Confocal_Images\CellposeData') # parent directory for saving masks
maskdir = outdir/'cell_nuc_mask'
maskdir.mkdir(exist_ok=True)

imlist = list(imdir.glob('*_CBS_*.tiff'))
imlist = sorted(imlist, key = lambda x: x.name)

#check if nvidia gpu card installed
gpucode =  os.system('nvidia-smi')
if gpucode ==0:
    gpu=True
else:
    gpu=False

celldiam = 310 # this number works well for HeLa cells, but should be adjusted based on results
nucdiam = 120 # this number works well for HeLa cells, but should be adjusted based on results

# for 'channels=' parameters 
#Cytoplasm model (‘cyto’) - the first channel is the channel to segment, and the second channel is an optional nuclear channel
# grayscale=0, R=1, G=2, B=3
cellchan = [3,1] 

cellprobthres = -1.4
flowthres = .9

for i in progressbar.progressbar(imlist):  
    im = tf.imread(i).max(axis=0)
    model = models.Cellpose(gpu=gpu, model_type='cyto')
    masks, *_ = model.eval(im, diameter=celldiam, channels=cellchan, cellprob_threshold= -1.4)
    cv2.imwrite(str(maskdir/f'cell_bw_{i.stem}.png'), masks.astype(np.uint16), params= [cv2.IMWRITE_PNG_COMPRESSION,9])
    covl = plot.mask_overlay(im.transpose(1,2,0),masks)
    cv2.imwrite(str(maskdir/f'ovl_cellmask_{i.stem}.png'), covl[...,::-1], params= [cv2.IMWRITE_PNG_COMPRESSION,9]) #cv2 expects BGR instead of RGB order
    
    #model (‘nuclei’) - set the first channel as 0=grayscale, 1=red, 2=green, 3=blue; and set the second channel to zero
    model = models.Cellpose(gpu=gpu, model_type='nuclei')
    masks, *_ = model.eval(im, diameter=nucdiam, channels=[1,0],flow_threshold=.9 )
    cv2.imwrite(str(maskdir/f'nuc_bw_{i.stem}.png'), masks.astype(np.uint16), params= [cv2.IMWRITE_PNG_COMPRESSION,9])
    novl = plot.mask_overlay(im[0],masks)
    cv2.imwrite(str(maskdir/f'ovl_nucmask_{i.stem}.png'), novl[...,::-1], params= [cv2.IMWRITE_PNG_COMPRESSION,9])

              

### Collect Cell and Nuc Ids to saved CSV Files

In [1]:
from skimage.measure import label, regionprops, regionprops_table
import imageio as io
import datetime as dt
import progressbar
import mahotas
import scipy.ndimage as ndi
import cupy as cp

In [106]:
class MaskMap():
    ''' Takes a mask image, labeled or unlabeled, with one or more objects and gets perimeter
    coordiantes or distance maps for particular objects based on either the whole object's pixel
    coordinates, [N,2] array, or the objects label number. '''
    def __init__(self,bw=None, coord=None, label=None):
        self.bw = bw
        self.coord = coord
        self.label = label
        self.getmaskimage()
        
    def getmaskimage(self):
        # recreate isolated single masks to get measurements relative to whole image
        if (self.bw is not None) & (self.coord is not None):
            labelbw = np.zeros_like(self.bw)
            labelbw[self.coord[:,0], self.coord[:,1]] = 1 
            self.labelbw = labelbw
        elif (self.bw is not None) & (self.label is not None):
            self.labelbw = np.where(self.bw==self.label,1,0)
            
        
    def getmaskdistmap(self, bw=None, vector=True):         
        ''' returns a vector with all pixel distances of a mask, obtained from a
        distance transform map. '''  

        dm = ndi.distance_transform_edt(bw)
        dmap= dm[bw.astype(bool)]
        if vector:
            return np.round(dmap,decimals=1).tolist()
        else:
            return dm
    
    def getperimcoords(self, bw):
        '''takes regionprops coords of a mask object associated with image 'bw'
        and generates coords (N,2) array (-> converted to a list) of just the outline of the object relative
        to image of size 'bw'. '''
       
        return np.argwhere(mahotas.bwperim(bw)).tolist() # conversion to list for easier restoration to numpy array after reimport where np arrays are now strings within a df
    
    def getnucpixmap(self, nucbw, cellbw, vector=True):
        maskinverted = (nucbw==0)*1 
        nucoutdist = ndi.distance_transform_edt(maskinverted) # distance map from outer nuclear edge towards cell edge 
        nucindist = ndi.distance_transform_edt(nucbw) #distances from within nuc to nuc edge
        nucdist = nucoutdist-nucindist # dist within nucleus becomes negative values and out of nuc are positive
        nucpixvector = nucdist[cellbw.astype(bool)]
        if vector:
            return np.round(nucpixvector, decimals=1).tolist()
        else:
            return nucdist


In [ ]:

imdir = Path(r"E:\Confocal_export_0421\ijstack")
maskdir = Path(r"E:\Ilastik test0521\cell_nuc_mask")
csvdir = Path(r"E:\Ilastik test\\Segmented_Centroids_CSVFiles")
outdir = csvdir/'cp_bw_measures1'
outdir.mkdir(exist_ok=True)

csvlist = list(csvdir.glob('*DonutBkg_*.csv'))
csvlist = sorted(csvlist, key= lambda x: x.name)
# print(len(csvlist))


def getimages(i:str):
    im = tf.imread(imdir/f'{i}.tiff')
    imax = im.max(axis=0)
    imax = imax.transpose((1,2,0))
    bwcell = io.imread(maskdir/f'cell_bw_{i}.png')
    bwnuc = io.imread(maskdir/f'nuc_bw_{i}.png')
    return im, imax, bwcell, bwnuc


outdfs = []
imgstats = []
structmaps = []
for csvitem in progressbar.progressbar(csvlist):
    df = pd.read_csv(csvitem, index_col=0)
    df['Label'] = df.Label.str.extract(r'MAX_(.*).tiff') # get pure image names here
    df['MeanSub'] = df['Mean'] - df['bkg_mean']
    df.loc[df.MeanSub<0,'MeanSub'] = 0
    imlist = df.Label.unique() # all unique pure image names
    # print(len(imlist))
    df['cellid'] = np.nan
    df['nucid'] = np.nan
    df['cytoid'] = np.nan
    

    for i in progressbar.progressbar(imlist):
        imgdict = {}
        im, imax, bwcell, bwnuc = getimages(i)
        imgdict['img_mean'] = im.mean(axis=(0,2,3))
        imgdict['img_min'] = im.min(axis=(0,2,3))
        imgdict['img_max'] = im.max(axis=(0,2,3))
        imgdict['img_median'] = np.median(im,axis=(0,2,3))
        imgdict['img_std'] = np.std(im,axis=(0,2,3))
        imgdict['ch'] = np.arange(1, im.shape[1]+1)
        imgdict['image'] = [i]* im.shape[1] # for creating a df, need matching length of image names to num of image channels
        imgstats.append(imgdict)
        
        bwcyto = np.logical_xor(bwcell, bwnuc)
        bwcyto = bwcyto.astype(np.uint8)
        
         # labeled cell binary used as look-up table for spot centroids.
        df.loc[df.Label == i,'cellid'] = bwcell[df.loc[df.Label == i,'Y'].astype(np.int32),df.loc[df.Label == i,'X'].astype(np.int32)]
        df.loc[df.Label == i,'nucid'] = bwnuc[df.loc[df.Label == i,'Y'].astype(np.int32),df.loc[df.Label == i,'X'].astype(np.int32)]
        df.loc[df.Label == i,'cytoid'] = bwcyto[df.loc[df.Label == i,'Y'].astype(np.int32),df.loc[df.Label == i,'X'].astype(np.int32)]
        props = ['label', 'area', 'centroid','eccentricity', 'major_axis_length','minor_axis_length', 'mean_intensity',
                'min_intensity', 'max_intensity','coords'] # coords should be indexed as col1, col0; to get image Y,X
        cellprop = regionprops_table(bwcell, intensity_image=imax, properties=props)
        nucprop =  regionprops_table(bwnuc, intensity_image=imax, properties=props)
        
        bwcell1 = np.stack([bwcell]*im.shape[0]) # replicate blobs to same image array shape without channel dim to measure over z stk
        im = im.transpose(0,2,3,1) # skimage needs channel on last dimmension
        cellprop1 = regionprops_table(bwcell1, im, properties=['intensity_image'])
        imgbkg = 1800
        tmpcalc = [cp.clip(cp.asarray(x)*1.0-imgbkg,0,65535) for x in cellprop1['intensity_image']] #after bkg subtraction, clipping faster than rescale image to rid neg values. Also use of cupy
        tmpcalc = cp.array([x.sum(axis=(0,1,2)) for x in tmpcalc]).get()
        
        for c in range(im.shape[3]):
            cellprop[f'zsum_intensity_{c}'] = tmpcalc[:,c]
      
        nucprop['cell_linkid']= bwcell[nucprop['centroid-0'].astype(np.uint32),nucprop['centroid-1'].astype(np.uint32)] # connect nuclei to their respective cells; regionprop centroid is Y,X (r,c)
        coordscell = []
        coordsnuc = []
        cellpixdist = []
        nucpixdist= []
        
        # get coords for just the blob perimeter instead of for the whole blob
        for idx, cd in enumerate(cellprop['coords']):
            cellmap = MaskMap(bwcell, cd)
            distdict = {}
            distdict['image']= i
            distdict['labelid'] = cellprop['label'][idx]
            distdict['struct'] = 'cell'
            coordscell.append(cellmap.getperimcoords(cellmap.labelbw))
            pixd = cellmap.getmaskdistmap(cellmap.labelbw)
            cellpixdist.append(pixd) 
        cellprop['coords'] = coordscell
        cellprop['cellpixdist'] = cellpixdist
        
        
        for idx, cd in enumerate(nucprop['coords']):
            nucmap = MaskMap(bwnuc, cd)
            distdictn = {}
            distdictn['image']= i
            distdictn['labelid'] = nucprop['label'][idx]
            distdictn['struct'] = 'nuc'
            tmpcellmask = MaskMap(bwcell, label=nucprop['cell_linkid'][idx]).labelbw #cell mask linked to each nuc
            coordsnuc.append(nucmap.getperimcoords(nucmap.labelbw))
            pixd = nucmap.getnucpixmap(nucmap.labelbw, tmpcellmask)
            nucpixdist.append(pixd)
        nucprop['coords'] = coordsnuc
        nucprop['nucpixdist'] = nucpixdist
        
        celldf = pd.DataFrame(cellprop)
        nucdf = pd.DataFrame(nucprop)
        celldf['struct'] = 'cell'
        celldf['image'] = i
        nucdf['struct'] = 'nuc'
        nucdf['image'] = i
        celldf.rename(columns={'label':'labelid',},inplace=True)
        nucdf.rename(columns={'label':'labelid'},inplace=True)
        outdfs.append(celldf)
        outdfs.append(nucdf)
        
    
    try:
        df = df.astype({'cellid': np.int32, 'nucid': np.int32, 'cytoid': np.int32})
    except Exception as e:
        print(e)
        
    stamp = dt.datetime.now().strftime('%y%m%d%H%M%S')
    df.rename(columns = {'Label':'image'},inplace=True)
    df.to_csv(outdir/f'cell_nucID_{stamp}_{csvitem.name}')

    
outdf = pd.concat(outdfs, ignore_index=True)
outdf.cell_linkid.fillna(-1, inplace=True)
outdf = outdf.astype({'cell_linkid':np.int32})
dfimages = pd.concat([pd.DataFrame(d) for d in imgstats], ignore_index=True)
outdf.to_csv(outdir/f'cpmask_data_{stamp}.csv')
dfimages.to_csv(outdir/f'WholeImage_Stats_{stamp}.csv')